**General description**

This notebook has been developed by Ana Teresa Lopez Jimenez @ LSHTM

It has been used in the preprint: High-content superresolution microscopy and deep learning assisted analysis reveals host and bacterial heterogeneity during Shigella infection. Ana T. López-Jiménez, Dominik Brokatzky, Kamla Pillay, Tyrese Williams, Gizem Özbaykal Güler and Serge Mostowy (2024)

This notebook allows to classify SETP7+ and SEPT7- associated bacteria.

**Importing packages:**


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

2.15.0


In [ ]:
!pip install pyyaml h5py

**Data loading from Google drive**


In [ ]:
from google.colab import drive
root = '/content/gdrive/'
drive.mount( root )

Mounted at /content/gdrive/


In [ ]:

import os
septin_dir_path = r'/My Drive/folder' # write here directory where files to be analysed are
os.makedirs(root+septin_dir_path, exist_ok=True)
os.listdir(root+septin_dir_path)


['single']

**Import the Model**

For this step, it is required to add the trained model "model_classification_SEPT7positive_vs_negative.hdf5" to Google drive, and specify the correct folder here below to retrieve it

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Activation, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization

In [ ]:

new_model = tf.keras.models.load_model('/content/gdrive/My Drive/folder/model_classification_SEPT7positive_vs_negative.hdf5') # Specify the folder containing the model
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 4)       112       
                                                                 
 batch_normalization (Batch  (None, 126, 126, 4)       16        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 4)         0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 8)         296       
                                                                 
 batch_normalization_1 (Bat  (None, 61, 61, 8)         32        
 chNormalization)                                                
                                                        

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.01)

new_model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
BATCH_SIZE = 32
IMG_SHAPE  = 128 # Our training data consists of images with width of 128 pixels and height of 128 pixels

In [ ]:
image_gen_val = ImageDataGenerator(
    rescale=1./255,
    fill_mode='nearest',)

val_data_gen = image_gen_val.flow_from_directory(batch_size=BATCH_SIZE,
                                                  directory=root+septin_dir_path,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='binary',

                                                 shuffle=False)

Found 486 images belonging to 1 classes.


**Applying the model and saving results**

In this step the model will be applied to generate predictions (0, 1). The results can be saved as a .txt document in the folder specificied here below.

In [ ]:
new_data = []
file_names = val_data_gen.filenames
predict_x=new_model.predict(val_data_gen)
classes_x=[1 * (x[0]>=0.5) for x in predict_x]
new_data = np.array([file_names, classes_x]).T
print(new_data)

np.savetxt('/content/gdrive/My Drive/folder/Results_septin.txt', new_data, fmt='%s') # Specify here the output folder


16/16 [==============================] - 3s 157ms/step
[['single/231011_p0042-01-Scene-01-P1-E02-AiPr-301n0.tif' '1']
 ['single/231011_p0042-01-Scene-01-P1-E02-AiPr-301n2.tif' '1']
 ['single/231011_p0042-01-Scene-01-P1-E02-AiPr-301n4.tif' '1']
 ['single/231011_p0042-01-Scene-02-P2-E02-AiPr-302n1.tif' '1']
 ['single/231011_p0042-01-Scene-02-P2-E02-AiPr-302n4.tif' '1']
 ['single/231011_p0042-01-Scene-02-P2-E02-AiPr-302n5.tif' '1']
 ['single/231011_p0042-01-Scene-02-P2-E02-AiPr-302n6.tif' '1']
 ['single/231011_p0042-01-Scene-03-P3-E02-AiPr-303n0.tif' '1']
 ['single/231011_p0042-01-Scene-03-P3-E02-AiPr-303n1.tif' '1']
 ['single/231011_p0042-01-Scene-03-P3-E02-AiPr-303n10.tif' '0']
 ['single/231011_p0042-01-Scene-03-P3-E02-AiPr-303n12.tif' '0']
 ['single/231011_p0042-01-Scene-03-P3-E02-AiPr-303n13.tif' '1']
 ['single/231011_p0042-01-Scene-03-P3-E02-AiPr-303n14.tif' '0']
 ['single/231011_p0042-01-Scene-03-P3-E02-AiPr-303n15.tif' '1']
 ['single/231011_p0042-01-Scene-03-P3-E02-AiPr-303n17.tif'